In [47]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [48]:
sa = gspread.service_account(filename='geprek-wani-c700b4b8002f.json')

In [49]:
sheet = sa.open('Wani Spreadsheet')

In [50]:
database = sheet.worksheet('Database Harga')

In [51]:
df_harga = pd.DataFrame(database.get_all_records())

In [52]:
worksheet = sheet.worksheet('Pesanan')

In [53]:
df = pd.DataFrame(worksheet.get_all_records())

In [54]:
df.rename(columns={'Request cabe GEPREK': 'Cabe'}, inplace=True)

In [55]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,14/04/2023 07:58:46,Rey,Indomie Telur Geprek,0,,Es teh,
1,14/04/2023 07:59:01,Rey,Ayam geprek tanpa nasi,1,,,
2,14/04/2023 08:34:02,Tot,Nasi Ayam Geprek,3,,Es teh,
3,14/04/2023 08:34:43,Tot,Nasi Ayam Bakar,,,Es teh,
4,14/04/2023 08:37:10,alvin,Mie Goreng Biasa,,Usus Crispy tanpa bumbu,,


In [56]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1
df[list] = df[list].replace('', 1)

df[list]

,Makanan Utama,Cabe,Nama
0,Indomie Telur Geprek,0,Rey
1,Ayam geprek tanpa nasi,1,Rey
2,Nasi Ayam Geprek,3,Tot
3,Nasi Ayam Bakar,1,Tot
4,Mie Goreng Biasa,1,alvin
5,Nasi Ayam Geprek,1,Max


In [91]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek', case=False) & ~df['Makanan Utama'].str.contains('Indomie')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
1,14/04/2023 07:59:01,Rey,Ayam geprek tanpa nasi,1,,,
2,14/04/2023 08:34:02,Tot,Nasi Ayam Geprek,3,,Es teh,
5,14/04/2023 08:53:14,Max,Nasi Ayam Geprek,1,,,


In [92]:
# Dataframe non-geprek
df_not_geprek = df[~df['Makanan Utama'].str.contains('geprek', case=False)]

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe indomie geprek
df_indomie_geprek = df[df['Makanan Utama'].str.contains('Indomie') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

In [94]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_indomie_geprek['Cabe'] == i)} {df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Nama'])})" for i in df_indomie_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-04-14 09:39:49.690822
1 Nasi Ayam Geprek Cabe 3 (Tot)
1 Nasi Ayam Geprek Cabe 1 (Max)
1 Ayam Geprek tanpa nasi Cabe 1 (Rey)
1 Nasi Ayam Bakar (Tot)
1 Mie Goreng Biasa (alvin)
1 Indomie Telur Geprek Cabe 0 (Rey)
1 Usus Crispy tanpa bumbu (alvin)
3 Es teh (Rey, Tot, Tot)


In [60]:
# remove all text after '(' in combined_prints
# combined_prints = [i.split('(')[0] for i in combined_prints]

# print('\n'.join(combined_prints))

In [61]:
# payload = {'message' : '\n'.join(combined_prints)}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload)